In [1]:
# Exercise 1

import random
import time
from bloom_filter import BloomFilter, BloomFilter_KM_Opt, BloomFilter_Uni_Hash, BloomFilter_QF

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
# Initialize Bloom Filter
bf = BloomFilter(n=10**7, f=0.02)

# Insert 10 million uniformly random integers
inserted_elements = [str(random.randint(0, 10**12)) for _ in range(10**7)]

# Execure add function for each element
for elem in inserted_elements:
    bf.add(elem)

# Save inserted elements to a file
with open('inserted_elements.txt', 'w') as f:
    for elem in inserted_elements:
        f.write(elem + '\n')

# Perform 10^6 random lookups from the universe
random_elements = [str(random.randint(0, 10**12)) for _ in range(10**6)]
false_positives = 0

start_time = time.time()
for elem in random_elements:
    if bf.query(elem):
        false_positives += 1
end_time = time.time()

print(f"False positive rate: {false_positives / 10**6}")
print(f"Time for 10^6 random lookups: {end_time - start_time} seconds")

# Perform 10^6 successful lookups
successful_lookups = random.sample(inserted_elements, 10**6)

start_time = time.time()
for elem in successful_lookups:
    bf.query(elem)
end_time = time.time()

print(f"Time for 10^6 successful lookups: {end_time - start_time} seconds")

False positive rate: 0.020709
Time for 10^6 random lookups: 4.3739800453186035 seconds
Time for 10^6 successful lookups: 16.355926990509033 seconds


In [3]:
# Exercise 2

n_original = 100000
f = 0.02

bf_original = BloomFilter(n=n_original, f=f)
print(f"Original Bloom filter size: {bf_original.m} bits")

n_larger = n_original*100
bf_larger = BloomFilter(n=n_larger, f=f)
print(f"Larger Bloom filter size: {bf_larger.m} bits")

n_smaller = n_original//100
bf_smaller = BloomFilter(n=n_smaller, f=f)
print(f"Smaller Bloom filter size: {bf_smaller.m} bits")

Original Bloom filter size: 814236 bits
Larger Bloom filter size: 81423633 bits
Smaller Bloom filter size: 8142 bits


In [4]:
# Exercise 3
import math
import mmh3
import bitarray
from nltk import ngrams

class BloomFilterPartition:
    def __init__(self, n: int, f: float):
        """
        Creates a Bloom Filter object.
        Args:
        n: max # of elements
        f: desired false positive rate
        """
        self.n = n
        self.f = f
        # Size of bit array
        self.m = int(-math.log(self.f) * self.n / (math.log(2)**2))
        # Number of hash functions
        self.k = int(self.m * math.log(2) / self.n)
        #self.k = k
        # the chunk size for the hash function
        self.chunk = self.m // self.k
        # number of bytes required to store bit array
        self.n_bytes = (self.m + 7) // 8
        # Create the bit array
        self.bit_array = bitarray.bitarray(self.n_bytes * 8)
        self.bit_array.setall(0)  # Set all bits to 0

    def add(self, item: str):
        """
        Adds an item to the bloom filter.
        Args:
        item: the string to add
        """
        tokens = item.lower().split()
        for i in range(self.k):
            for n in range(1, 4): 
                n_grams = ngrams(tokens, n)
                for piece in n_grams:
                    index = mmh3.hash(" ".join(piece), i) % self.chunk
                    chunk_index = i * self.chunk + index
                    self.bit_array[chunk_index] = 1

    def query(self, item: str) -> bool:
        """
        Checks if an item might be in the bloom filter (may have false positives).
        Args:
        item: the string to query
        Returns:
        bool: True if item might be in the bloom filter, False if it definitely isn't
        """
        tokens = item.lower().split()
        for i in range(self.k):
            for n in range(1, 4):
                n_grams = ngrams(tokens, n)
                for piece in n_grams:
                    index = mmh3.hash(" ".join(piece), i) % self.chunk
                    chunk_index = i * self.chunk + index
                    if not self.bit_array[chunk_index]:
                        return False
        return True
    


In [5]:
# Initialize Bloom Filter
bp = BloomFilterPartition(n=10**7, f=0.02)

# Insert 10 million uniformly random integers
inserted_elements = [str(random.randint(0, 10**12)) for _ in range(10**7)]

# Execure add function for each element
for elem in inserted_elements:
    bp.add(elem)

# # Save inserted elements to a file
# with open('inserted_elements.txt', 'w') as f:
#     for elem in inserted_elements:
#         f.write(elem + '\n')

# Perform 10^6 random lookups from the universe
random_elements = [str(random.randint(0, 10**12)) for _ in range(10**6)]
false_positives = 0

start_time = time.time()
for elem in random_elements:
    if bp.query(elem):
        false_positives += 1
end_time = time.time()

print(f"False positive rate: {false_positives / 10**6}")
print(f"Time for 10^6 random lookups: {end_time - start_time} seconds")

# Perform 10^6 successful lookups
successful_lookups = random.sample(inserted_elements, 10**6)

start_time = time.time()
for elem in successful_lookups:
    bp.query(elem)
end_time = time.time()

print(f"Time for 10^6 successful lookups: {end_time - start_time} seconds")

False positive rate: 0.020316
Time for 10^6 random lookups: 4.462555885314941 seconds
Time for 10^6 successful lookups: 16.634416103363037 seconds


In [6]:
# Exercise 3 - Improvement 1 - KM Optimization

# Initialize Bloom Filter
bKM = BloomFilter_KM_Opt(n=10**7, f=0.02)

# Insert 10 million uniformly random integers
inserted_elements = [str(random.randint(0, 10**12)) for _ in range(10**7)]

# Execure add function for each element
for elem in inserted_elements:
    bKM.add(elem)

# # Save inserted elements to a file
# with open('inserted_elements.txt', 'w') as f:
#     for elem in inserted_elements:
#         f.write(elem + '\n')

# Perform 10^6 random lookups from the universe
random_elements = [str(random.randint(0, 10**12)) for _ in range(10**6)]
false_positives = 0

start_time = time.time()
for elem in random_elements:
    if bKM.query(elem):
        false_positives += 1
end_time = time.time()

print(f"False positive rate: {false_positives / 10**6}")
print(f"Time for 10^6 random lookups: {end_time - start_time} seconds")

# Perform 10^6 successful lookups
successful_lookups = random.sample(inserted_elements, 10**6)

start_time = time.time()
for elem in successful_lookups:
    bKM.query(elem)
end_time = time.time()

print(f"Time for 10^6 successful lookups: {end_time - start_time} seconds")

False positive rate: 0.020043
Time for 10^6 random lookups: 0.7476139068603516 seconds
Time for 10^6 successful lookups: 1.6923739910125732 seconds


In [8]:
# Exercise 3 - Improvement 2 - Universal Hashing

# Initialize Bloom Filter
bUH = BloomFilter_Uni_Hash(n=10**7, f=0.02)

# Insert 10 million uniformly random integers
inserted_elements = [str(random.randint(0, 10**12)) for _ in range(10**7)]

# Execure add function for each element
for elem in inserted_elements:
    bUH.add(elem)

# # Save inserted elements to a file
# with open('inserted_elements.txt', 'w') as f:
#     for elem in inserted_elements:
#         f.write(elem + '\n')

# Perform 10^6 random lookups from the universe
random_elements = [str(random.randint(0, 10**12)) for _ in range(10**6)]
false_positives = 0

start_time = time.time()
for elem in random_elements:
    if bUH.query(elem):
        false_positives += 1
end_time = time.time()

print(f"False positive rate: {false_positives / 10**6}")
print(f"Time for 10^6 random lookups: {end_time - start_time} seconds")

# Perform 10^6 successful lookups
successful_lookups = random.sample(inserted_elements, 10**6)

start_time = time.time()
for elem in successful_lookups:
    bUH.query(elem)
end_time = time.time()

print(f"Time for 10^6 successful lookups: {end_time - start_time} seconds")

False positive rate: 0.020468
Time for 10^6 random lookups: 0.7106449604034424 seconds
Time for 10^6 successful lookups: 1.7898399829864502 seconds


In [2]:
# Exercise 3 - Improvement 3 - Quotient Filter

# Initialize Bloom Filter
bQF = BloomFilter_QF(n=10**7, f=0.02)

# Insert 10 million uniformly random integers
inserted_elements = [str(random.randint(0, 10**12)) for _ in range(10**7)]

# Execure add function for each element
for elem in inserted_elements:
    bQF.add(elem)

# # Save inserted elements to a file
# with open('inserted_elements.txt', 'w') as f:
#     for elem in inserted_elements:
#         f.write(elem + '\n')

# Perform 10^6 random lookups from the universe
random_elements = [str(random.randint(0, 10**12)) for _ in range(10**6)]
false_positives = 0

start_time = time.time()
for elem in random_elements:
    if bQF.query(elem):
        false_positives += 1
end_time = time.time()

print(f"False positive rate: {false_positives / 10**6}")
print(f"Time for 10^6 random lookups: {end_time - start_time} seconds")

# Perform 10^6 successful lookups
successful_lookups = random.sample(inserted_elements, 10**6)

start_time = time.time()
for elem in successful_lookups:
    bQF.query(elem)
end_time = time.time()

print(f"Time for 10^6 successful lookups: {end_time - start_time} seconds")

False positive rate: 0.002222
Time for 10^6 random lookups: 0.47637081146240234 seconds
Time for 10^6 successful lookups: 0.6565229892730713 seconds
